In [ ]:
import urllib2
import json
import sys
import time
import math
import datetime
import pickle
import geojson
from geojson import Feature, Point, FeatureCollection

In [ ]:
cached_responses = {}

In [ ]:
def get_api_key():
    return "YOUR-API-KEY"

In [ ]:
def get_url_from_lat_lon(latitude, longitude):
    url = 'http://api.wunderground.com/api/'+ get_api_key() + '/geolookup/q/' + str(latitude) + ',' + str(longitude) + '.json'
    return url

In [ ]:
def get_neighbors(url):
    """ get the list of nearby airport and pws weather stations """
    f = urllib2.urlopen(url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    airports = parsed_json['location']['nearby_weather_stations']['airport']['station']
    airports = map(lambda x: x['icao'], airports)
    airports = filter(len, airports)
    icaos = map(lambda x: ("http://api.wunderground.com/api/" + get_api_key() + "/conditions/q/" + x + ".json"), airports)
    pws = parsed_json['location']['nearby_weather_stations']['pws']['station']
    ids = map(lambda x: ("http://api.wunderground.com/api/" + get_api_key() + "/conditions/q/pws:" + x['id'] + '.json'), pws)
    stations = icaos + ids
    return stations

In [ ]:
def get_feature(station_url):
    time.sleep(20)
    print station_url
    f = urllib2.urlopen(station_url)
    json_string = f.read()
    parsed_json = json.loads(json_string)
    lat = float(parsed_json['current_observation']['display_location']['latitude'] )
    lon = float(parsed_json['current_observation']['display_location']['longitude'] )
    zip = parsed_json['current_observation']['display_location']['zip']
    weather = parsed_json['current_observation']['weather']
    temperature_string = parsed_json['current_observation']['temperature_string']
    icon = parsed_json['current_observation']['icon_url']
    point = Point((lat, lon))
    prop  = {'weather' : weather, 'temperature_string' : temperature_string, 'icon' : icon}
    f = Feature(geometry=point, properties=prop)
    cached_responses[station_url] = f
    return f

In [ ]:
def get_feature_collection(lat, lon):
    url = get_url_from_lat_lon(lat, lon)
    print url
    stations = get_neighbors(url)
    print stations
    features = []
    for station_url in stations:
        if station_url in cached_responses:
            print "cached response"
            f = cached_responses[station_url]
        else:
            print "not cached response"
            f = get_feature(station_url)
        features.append(f)
    collection = FeatureCollection(features)
    print collection
    return collection

In [ ]:
get_feature_collection(34.776,-122.395)
print cached_responses